In [1]:
!pip install llama-index
!pip install slack_sdk
!pip install qdrant_client


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

from dotenv import find_dotenv, load_dotenv
from llama_index import GPTListIndex, SlackReader

import logging
import structlog
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"), override=True)

/Users/alfie/Documents/code/hash/apps/hash-agents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.util.retry").setLevel(logging.CRITICAL)
logging.getLogger("openai").setLevel(logging.CRITICAL)
logging.getLogger("httpcore").setLevel(logging.CRITICAL)

In [5]:
import json
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

embedding_model = "text-embedding-ada-002"

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10))
def create_embedding_with_backoff(**kwargs):
    return openai.Embedding.create(**kwargs)

def create_embedding(message, num: int | None, total: int | None):
    logger.debug("Creating embedding", num=num, total=total)
    embedding = create_embedding_with_backoff(
        input=json.dumps(message),
        model=embedding_model,
    )["data"][0]["embedding"]
    return {
        "embedding": embedding,
        "message": message,
    }

In [6]:
from collections import namedtuple
from queue import Queue

from attr import dataclass
from notion_client import Client
from notion_client.helpers import collect_paginated_api

NotionBlockResult = namedtuple(
    "NotionBlockResult",
    ["content_components", "sub_pages"],
)

@dataclass
class NotionBlockContent:
    contents: str

@dataclass
class NotionBlockIdentifier:
    identifier: str

NotionBlockContentOrIdentifier = NotionBlockContent | NotionBlockIdentifier

def traverse_notion_block(
    notion_client: Client,
    task_queue: Queue,
    block_id,
) -> NotionBlockResult:
    print(f"Getting children blocks of {block_id}")

    block_children = collect_paginated_api(
        notion_client.blocks.children.list,
        block_id=block_id,
    )

    sub_pages = []
    content_components: list[NotionBlockContentOrIdentifier] = []

    for block_child in block_children:
        result_obj = block_child[block_child["type"]]

        if "rich_text" in result_obj:
            for rich_text in result_obj["rich_text"]:
                # skip if doesn't have text object
                if "text" in rich_text:
                    text = rich_text["text"]["content"]
                    content_components.append(NotionBlockContent(text))
                elif "plain_text" in rich_text:
                    text = rich_text["plain_text"]
                    content_components.append(NotionBlockContent(text))
        elif "title" in result_obj:
            text = result_obj["title"]
            content_components.append(NotionBlockContent(text))
        if block_child["has_children"]:
            task_queue.put(block_child["id"])
            if block_child["type"] == "child_page":
                sub_pages.append(block_child["id"])
            else:
                content_components.append(NotionBlockIdentifier(block_child["id"]))

    return NotionBlockResult(content_components=content_components, sub_pages=sub_pages)

In [7]:
import concurrent.futures
import os
import queue
import threading

from notion_client import Client

integration_token = os.getenv("NOTION_API_KEY")
page_ids = ["52b92f24e14b43be83f64c206f211413"]

notion_client = Client(auth=integration_token)

executor = concurrent.futures.ThreadPoolExecutor()
task_queue = queue.Queue()
results = {}


def process_queue():
    while True:
        block_id = task_queue.get()
        results[block_id] = executor.submit(
            traverse_notion_block,
            notion_client,
            task_queue,
            block_id,
        )
        task_queue.task_done()


queue_thread = threading.Thread(target=process_queue)
queue_thread.start()

for page_id in page_ids:
    task_queue.put(page_id)

task_queue.join()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/alfie/Documents/code/hash/apps/hash-agents/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
load_verify_locations cafile='/Users/alfie/Documents/code/hash/apps/hash-agents/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
Getting children blocks of 52b92f24e14b43be83f64c206f211413
INFO:httpx:HTTP Request: GET https://api.notion.com/v1/blocks/52b92f24e14b43be83f64c206f211413/children "HTTP/1.1 200 OK"
HTTP Request: GET https://api.notion.com/v1/blocks/52b92f24e14b43be83f64c206f211413/children "HTTP/1.1 200 OK"
Getting children blocks of 3fd993fa-4202-40fb-967f-ce7d80784e3bGetting children blocks of 332983e9-93a8-4a1d-be0c-877dc4c64d39

Getting children blocks of 612b1eb3-dff6-49bd-88dd-0484c7b8a4d3
Getting children blocks of 20387f14-fbc7-475c-90ea-026bc6584310
Getting children blocks o

In [8]:
results

{'52b92f24e14b43be83f64c206f211413': <Future at 0x11d3c7ed0 state=finished returned NotionBlockResult>,
 '3fd993fa-4202-40fb-967f-ce7d80784e3b': <Future at 0x103cc1050 state=finished returned NotionBlockResult>,
 '332983e9-93a8-4a1d-be0c-877dc4c64d39': <Future at 0x11d3c7090 state=finished returned NotionBlockResult>,
 '612b1eb3-dff6-49bd-88dd-0484c7b8a4d3': <Future at 0x11d3bcf90 state=finished returned NotionBlockResult>,
 '20387f14-fbc7-475c-90ea-026bc6584310': <Future at 0x11f904390 state=finished returned NotionBlockResult>,
 '6954ab5c-8bbc-4dec-afe0-618de444a797': <Future at 0x11f906550 state=finished returned NotionBlockResult>,
 '76422395-5ea5-4050-aa48-32ab29b0ac31': <Future at 0x11f907610 state=finished returned NotionBlockResult>,
 '49c3fb85-5a3b-4fe2-8e1d-aeb8793701bf': <Future at 0x11d3c6450 state=finished returned NotionBlockResult>,
 '36fc4ecb-feaa-4e88-8dcb-4eab08f9cc99': <Future at 0x103c17450 state=finished returned NotionBlockResult>,
 '0e19c419-2a41-423d-a03f-130b00

In [15]:
pages_to_explore = [*page_ids]
explored_pages = set()
pages_to_contents = {}


def collapse_text(root_block_id, depth):
    block_result = results[root_block_id].result()
    pages_to_explore.extend(block_result.sub_pages)

    block_contents = []
    for content_component in block_result.content_components:
        if isinstance(content_component, NotionBlockIdentifier):
            block_contents.append(collapse_text(content_component.identifier, depth + 1))
        else:
            block_contents.append(content_component.contents)

    indent = (depth * "\t")
    return indent + ("\n" + indent).join(block_contents)

while len(pages_to_explore) > 0:
    page_id = pages_to_explore.pop(0)
    if page_id in explored_pages:
        continue
    explored_pages.add(page_id)

    print(f"Collapsing text for page {page_id}")
    pages_to_contents[page_id] = collapse_text(page_id, 0)

Collapsing text for page 52b92f24e14b43be83f64c206f211413
Collapsing text for page 3fd993fa-4202-40fb-967f-ce7d80784e3b
Collapsing text for page 332983e9-93a8-4a1d-be0c-877dc4c64d39
Collapsing text for page 612b1eb3-dff6-49bd-88dd-0484c7b8a4d3
Collapsing text for page 36fc4ecb-feaa-4e88-8dcb-4eab08f9cc99
Collapsing text for page 49c3fb85-5a3b-4fe2-8e1d-aeb8793701bf


In [16]:
pages_to_contents

{'52b92f24e14b43be83f64c206f211413': 'Subpage\nTest Heading 1\nHeading 1 Contents\nHeading 1 Subpage\nTest Heading 2\nHeading 2 Contents\nHeading 2 Subpage\nTest Toggle Heading 1\n\tToggle Heading 1 Contents\n\tToggle Heading 1 Subpage\n\tTest Toggle Heading 2\n\t\t\tToggle Heading 2 Contents\nBullet Point\nBullet Point 2\n\tNested Bullet Point 2\nCallout contents\n\tHeading in Callout\n\tHeading in Callout contents\n\tPage in Callout',
 '3fd993fa-4202-40fb-967f-ce7d80784e3b': 'Subpage Contents',
 '332983e9-93a8-4a1d-be0c-877dc4c64d39': 'Heading 1 Subpage',
 '612b1eb3-dff6-49bd-88dd-0484c7b8a4d3': 'Heading 2 Subpage Contents',
 '36fc4ecb-feaa-4e88-8dcb-4eab08f9cc99': 'Toggle Heading 1 Subpage Contents',
 '49c3fb85-5a3b-4fe2-8e1d-aeb8793701bf': 'Page in Callout Contents'}

In [ ]:
def create_payload(document):
    return {
        "text": document.text if document.text else None,
        "doc_id": document.doc_id if document.doc_id else None,
        "extra_info": document.extra_info if document.extra_info else None
    }

In [ ]:
embedding = create_embedding(message=create_payload(documents[0]), num=0, total=1)

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    embeddings = list(
        executor.map(
            lambda idx_item: create_embedding(
                message=create_payload(idx_item[1]),
                num=idx_item[0],
                total=len(documents),
            ),
            enumerate(documents),
        ),
    )

In [ ]:
from llama_index import LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
prompt_helper = PromptHelper.from_llm_predictor(llm_predictor=llm_predictor)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    chunk_size_limit=1024,
    prompt_helper=prompt_helper,
)

In [ ]:
from llama_index import GPTQdrantIndex
import qdrant_client

client = qdrant_client.QdrantClient(
    url="http://localhost:6333"
)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("My docker-compose isn't working, how can I fix it?")
response

In [ ]:
response.response